In [157]:
# re-import modules without restarting kernel
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


from io_helpers import *
from matching_helpers import *

#essential imports
import pandas as pd
import numpy as np

#date
import datetime as dt

from IPython.display import clear_output







nb_controls_per_subject = 4



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Matching movies and dead actors/directors

### 1) Load and clean the meta_movies data generated with Amazon API

In [6]:
## open datafiles
##

features_meta = ['asin', 'actors', 'directors', 'creators', 'authors']

MMTV = pd.read_csv('DATA/metadata_processed/meta_Movies_and_TV_processed(v1).csv', low_memory=False)[features_meta]
AIV = pd.read_csv('DATA/metadata_processed/meta_Amazon_Instant_Video_processed(v1).csv', low_memory=False)[features_meta]

video_df = pd.concat((MMTV, AIV))
video_df = df_rm_punctuation(video_df)

In [7]:
assert(MMTV.shape[0]+AIV.shape[0] == video_df.shape[0])
video_df.head()

,asin,actors,directors,creators,authors
0,0000143561,,,,
1,0000589012,,,,Joe Herzanek
2,0000695009,,,,Epilepsy Foundation
3,000107461X,Douglas Miller,,"(Joe Thomas,Producer),(Joseph Williams,Producer)",
4,0000143529,Alton Brown,,,


## 2) Clean and filter dead actors data

In [170]:
deaths = pd.read_csv('DATA/deaths.csv', low_memory=False, encoding="ISO-8859-1")
dead_actors = deaths[deaths['Actor']==True].drop('Actor', axis=1).drop('Author', axis=1).drop('Musician', axis=1).drop('Description', axis=1)
# for some reason the index skips some numbers
dead_actors = dead_actors.reset_index().drop('index', axis=1)
dead_actors['Name'] = std_actors_names(dead_actors['Name'])
dead_actors.head()

,Name,Birth Date,Death Date
0,jack weston,1924,1996-05-03
1,john beradino,1917,1996-05-19
2,jon pertwee,1919,1996-05-20
3,lash larue,1917,1996-05-21
4,enrique álvarez félix,1934,1996-05-24


## 3) Build (metadata<->actors) controls list

In [9]:
# filter out blanks, products_actors will be re-used
amazon_actor_products = video_df[(video_df["actors"].isnull()==False)]
amazon_actor_products = amazon_actor_products[amazon_actor_products['actors']!='']
print(amazon_actor_products.shape)
amazon_actor_products.head()

(144720, 5)


,asin,actors,directors,creators,authors
3,000107461X,Douglas Miller,,"(Joe Thomas,Producer),(Joseph Williams,Producer)",
4,0000143529,Alton Brown,,,
5,0000143502,Alton Brown,,,
6,0000143588,Ina Garten,,,
11,0001485423,Sr Faustina,J Paddy Nolan,,


In [16]:
amazon_actor_names = [troup.split(',') for troup in list(amazon_actor_products['actors'])]
amazon_actor_names = [lm.lower() for lm in np.concatenate(amazon_actor_names)]
amazon_actor_names = list(set(amazon_actor_names))

# weird... 
print(amazon_actor_names[:5])
amazon_actor_names = amazon_actor_names[:1]+ amazon_actor_names[2:]

amazon_actors = pd.DataFrame(amazon_actor_names, columns=['Name'])
amazon_actors['Name'] = std_actors_names(amazon_actors['Name'])
amazon_actors['Birth Date'] = pd.Series()
amazon_actors['Death Date'] = pd.Series()
amazon_actors.head()

['dr lisa schirch', '', 'cortknee', 'britton crosley', 'prof joseph stiglitz']


,Name,Birth Date,Death Date
0,dr lisa schirch,NaN,NaN
1,cortknee,NaN,NaN
2,britton crosley,NaN,NaN
3,prof joseph stiglitz,NaN,NaN
4,robert harrison,NaN,NaN


In [171]:
## filter out deads
##

dead_actors['clname'] = dead_actors['Name'].map(cleanstr)
dead_actors = dead_actors.drop('Name', axis=1)
nondead_actors = pd.DataFrame(list(map(lambda s : cleanstr(s), amazon_actor_names)),columns=['clname'])
nondead_actors= nondead_actors[nondead_actors['clname'].map(lambda e : len(e.split())>1)]

for idx, name in enumerate(dead_actors['clname'].tolist()):
    if idx % 100 == 0 : 
        print(idx)
    mapp = nondead_actors['clname'].str.contains(name)
    if mapp.any():
        nondead_actors = nondead_actors[mapp==False] 

nondead_actors['Birth Date'] = pd.Series()
nondead_actors['Death Date'] = pd.Series()
nondead_actors.head()

controls = nondead_actors

all_actors = pd.concat((dead_actors, nondead_actors))

0
100
200
300
400
500
600


In [172]:
controls.shape, dead_actors.shape, amazon_actors.shape, amazon_actors.shape[0]-controls.shape[0]

((149443, 3), (639, 3), (155561, 3), 6118)

## 4) Match dead actors and meta data

In [101]:
pd.options.mode.chained_assignment = None 
meta_dead_actors = pd.DataFrame()

amazon_actor_products['clname'] = amazon_actor_products['actors'].map(cleanstr)

for idx, actor in enumerate(list(dead_actors['clname'])):
    
    match = amazon_actor_products[amazon_actor_products['clname'].str.contains(actor)==True].drop('actors',axis=1)
    match['clname'] = actor
    match['Death Date'] = dead_actors.reset_index()['Death Date'].get_value(idx)
    meta_dead_actors = meta_dead_actors.append(match)
    
    if idx%100==0 and idx!=0: 
        print('actor {} in {} - Done'.format(idx, len(list(dead_actors['clname']))))
        
print(meta_dead_actors.shape)        
meta_dead_actors.head()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


actor 100 in 639 - Done
actor 200 in 639 - Done
actor 300 in 639 - Done
actor 400 in 639 - Done
actor 500 in 639 - Done
actor 600 in 639 - Done
(14282, 6)


,asin,directors,creators,authors,clname,Death Date
3487,0792840461,Norman Jewison,"(Haskell Wexler,Cinematographer),(Norman Jewis...",,jack weston,1996-05-03
10823,6300216217,,,,jack weston,1996-05-03
11212,6300270297,Richard Lester,"(Paul Wilson,Cinematographer),(John Bloom,Edit...",,jack weston,1996-05-03
11233,6300270025,Brian G Hutton,"(Andre Morgan,Producer),(Daniel Grodnik,Produc...",,jack weston,1996-05-03
12506,6301514017,Stuart Rosenberg,,,jack weston,1996-05-03


In [155]:
len(set(meta_dead_actors['clname']))

481

## 5) Subsample control actors and match meta data

In [173]:
from numpy.random import randint

def subsample_df (df, nbWanted) : 
    assert nbWanted < len(df)
    randomIndex = randint(0, len(df)-1, nbWanted)
    return df.loc[randomIndex]

sub_controls = subsample_df(controls, nb_controls_per_subject*len(dead_actors))
print(sub_controls.shape)
sub_controls.head(3)

(2556, 3)


/usr/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


,clname,Birth Date,Death Date
21051,john colton,NaN,NaN
45953,pauline moran,NaN,NaN
96531,yo hitoto,NaN,NaN


In [175]:
pd.options.mode.chained_assignment = None 
meta_control_actors = pd.DataFrame()

d=''
for idx, actor in enumerate(list(sub_controls.dropna(how='all')['clname'])):  
    match = pd.DataFrame()
    match = amazon_actor_products[amazon_actor_products['clname'].str.contains(actor)==True].drop('actors',axis=1)
    match['clname'] = actor
    match['Death Date'] = sub_controls.reset_index()['Death Date'].get_value(idx)
    meta_control_actors = meta_control_actors.append(match)
    
    if idx%100==0 and idx!=0: 
        print('actor {} in {} - Done'.format(idx, len(list(sub_controls['clname']))))


print(meta_control_actors.shape)
print(len(set(meta_dead_actors['clname'])))
meta_control_actors.head()

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


actor 100 in 2556 - Done
actor 200 in 2556 - Done
actor 300 in 2556 - Done
actor 400 in 2556 - Done
actor 500 in 2556 - Done
actor 600 in 2556 - Done
actor 700 in 2556 - Done
actor 800 in 2556 - Done
actor 900 in 2556 - Done
actor 1000 in 2556 - Done
actor 1100 in 2556 - Done
actor 1200 in 2556 - Done
actor 1300 in 2556 - Done
actor 1400 in 2556 - Done
actor 1500 in 2556 - Done
actor 1600 in 2556 - Done
actor 1700 in 2556 - Done
actor 1800 in 2556 - Done
actor 1900 in 2556 - Done
actor 2000 in 2556 - Done
actor 2100 in 2556 - Done
actor 2200 in 2556 - Done
actor 2300 in 2556 - Done
actor 2400 in 2556 - Done
(7502, 6)
481


,asin,directors,creators,authors,clname,Death Date
160751,B003ABPU10,"Jacob Freydont-Attie,Joshua Milrad","(Marco Fargnoli,Cinematographer),(Joshua Milra...",,john colton,NaN
930,0767028627,,,,pauline moran,NaN
6696,1569383618,,,,pauline moran,NaN
15690,6302199034,,,,pauline moran,NaN
19482,6302925525,Herbert Wise,"(Michael Davis,Cinematographer),(Laurence Méry...",,pauline moran,NaN


## 6) Concatenate (metadatas<->artist) for dead and control artists

In [176]:
meta_all_actors = pd.concat((meta_dead_actors, meta_control_actors))

## 7) Loading Reviews data for Movies and TV and cleaning

In [117]:
f_MTV = 'DATA/review/reviews_Movies_and_TV.json.gz'
f_AIV = 'DATA/review/reviews_Amazon_Instant_Video.json.gz'
features_reviews = ['asin', 'reviewTime']
#features_reviews = ['asin', 'reviewTime', 'overall', 'helpful', 'summary', 'reviewText']

In [118]:
#b=gz_to_dataframe(f_AIV)[features_reviews]
#a=gz_to_dataframe(f_MTV)[features_reviews]
#safe_reviews_df = pd.concat((a, b))

In [119]:
reviews_df = safe_reviews_df
print(reviews_df.shape)
reviews_df.head()

(5190980, 6)


,asin,helpful,overall,reviewTime,summary,reviewText
0,0000143502,"[0, 0]",5.0,"01 17, 2013",Alton... nough said,This has some great tips as always and is help...
1,0000143529,"[0, 0]",5.0,"10 2, 2013",Ah Alton...,This is a great pastry guide. I love how Alto...
2,0000143561,"[2, 4]",2.0,"07 17, 2008",Don't waste your money,I have to admit that I am a fan of Giada's coo...
3,0000143588,"[9, 9]",5.0,"03 13, 2009",VOLUME 1 & VOLUME 2-BETTER THAN THERAPY,I bought these two volumes new and spent over ...
4,0000143588,"[1, 4]",5.0,"01 18, 2009",Barefoot Contesst Vol 2,I am very pleased with the dvd only wish i cou...


In [120]:
def prefilter_reviews(review_df):
    filtered_reviews = review_df[review_df['asin'].isin(list(meta_all_actors['asin']))]
    filtered_reviews = filtered_reviews[features_reviews]
    filtered_reviews['reviewTime'] = list(pd.to_datetime(filtered_reviews['reviewTime'].str.replace(',','').str.replace(' ','-')))
    
    return filtered_reviews

In [132]:
filtered_reviews = prefilter_reviews(reviews_df)
filtered_reviews = filtered_reviews.reset_index()
print(filtered_reviews.shape)
filtered_reviews.head()

(665554, 3)


,index,asin,reviewTime
0,697,0101529376,2014-07-13
1,746,0307141985,2010-11-08
2,747,0307141985,2008-11-25
3,748,0307141985,2008-11-17
4,749,0307141985,2004-11-13


## 8) Matching meta/dead data with review data

In [ ]:
print(meta_dead_actors.shape)
meta_dead_actors.head()

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

def matching_meta_reviews(filtered_reviews, meta):
    
    processed_reviews = pd.DataFrame()
    
    for idx, asin in enumerate(meta['asin']):
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin]

        match_reviews['actor name'] = meta.reset_index()['clname'].get_value(idx)
        match_reviews['death date'] = meta.reset_index()['Death Date'].get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0 and idx!=0: 
            print('{} in {}'.format(idx, len(meta['asin'])))
        
    return processed_reviews

In [ ]:
deads_reviews = matching_meta_reviews(filtered_reviews, meta_dead_actors)
print(deads_reviews.shape)
deads_reviews.head()

## 8) Matching controls and reviews

In [177]:
print(meta_control_actors.shape)
meta_control_actors.head()

(7502, 6)


,asin,directors,creators,authors,clname,Death Date
160751,B003ABPU10,"Jacob Freydont-Attie,Joshua Milrad","(Marco Fargnoli,Cinematographer),(Joshua Milra...",,john colton,NaN
930,0767028627,,,,pauline moran,NaN
6696,1569383618,,,,pauline moran,NaN
15690,6302199034,,,,pauline moran,NaN
19482,6302925525,Herbert Wise,"(Michael Davis,Cinematographer),(Laurence Méry...",,pauline moran,NaN


In [178]:
meta_control_actors[meta_control_actors['clname'].map(lambda e : len(e.split())<=1)]

,asin,directors,creators,authors,clname,Death Date


In [179]:
def match_control_meta_reviews(filtered_reviews, meta):
    
    processed_reviews = pd.DataFrame()
    
    for idx, asin in enumerate(meta['asin']):
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin]

        match_reviews['actor name'] = meta.reset_index()['clname'].get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0 and idx!=0: 
            print('{} in {}'.format(idx, len(meta['asin'])))
        
    return processed_reviews

In [180]:
ctrls_reviews = match_control_meta_reviews(filtered_reviews, meta_control_actors)

/usr/lib/python3.6/site-packages/ipykernel/__main__.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


1000 in 7502
2000 in 7502
3000 in 7502
4000 in 7502
5000 in 7502
6000 in 7502
7000 in 7502


In [184]:
ctrls_reviews = processed_reviews_ctrls

## Save files

In [182]:
deads_reviews.to_csv('DATA/review_filtered/movie_reviews_matched.csv')

In [185]:
ctrls_reviews.to_csv('DATA/review_filtered/movie_reviews_matched_ctrls.csv')

In [192]:
ctrls_reviews.groupby('actor name').count().sort_values(by='index', ascending=False)[:10]

,index,asin,reviewTime
actor name,,,
patricia heaton,1801,1801,1801
frankie faison,1581,1581,1581
richard chamberlain,1389,1389,1389
kirsten dunst,1312,1312,1312
danny devito,1236,1236,1236
johnny depp,1220,1220,1220
amy yasbeck,1141,1141,1141
anthony zerbe,1039,1039,1039
rene russo,1008,1008,1008


In [210]:
### PROBLEM
###

#ctrls_reviews[ctrls_reviews['actor name']=='johnny depp'].groupby('asin').count()['asins']
idx = meta_control_actors[meta_control_actors['clname']=='johnny depp'].groupby('asin').count().reset_index()['asin']
reviews_df[reviews_df['asin'].isin(idx)].count()

asin          20095
helpful       20095
overall       20095
reviewTime    20095
summary       20095
reviewText    20095
dtype: int64